In [1]:
# Import libraries
import pandas as pd
import re
import requests as req
from bs4 import BeautifulSoup as bs
import json

# Importar datos

In [2]:
# Import data
df=pd.read_csv("../data/Artworks.csv")

df

,Title,Artist,ConstituentID,ArtistBio,Nationality,BeginDate,EndDate,Gender,Date,Medium,...,ThumbnailURL,Circumference (cm),Depth (cm),Diameter (cm),Height (cm),Length (cm),Weight (kg),Width (cm),Seat Height (cm),Duration (sec.)
0,"Ferdinandsbrücke Project, Vienna, Austria, Ele...",Otto Wagner,6210,"(Austrian, 1841–1918)",(Austrian),(1841),(1918),(Male),1896,Ink and cut-and-pasted painted pages on paper,...,http://www.moma.org/media/W1siZiIsIjU5NDA1Il0s...,NaN,NaN,NaN,48.6000,NaN,NaN,168.9000,NaN,NaN
1,"City of Music, National Superior Conservatory ...",Christian de Portzamparc,7470,"(French, born 1944)",(French),(1944),(0),(Male),1987,Paint and colored pencil on print,...,http://www.moma.org/media/W1siZiIsIjk3Il0sWyJw...,NaN,NaN,NaN,40.6401,NaN,NaN,29.8451,NaN,NaN
2,"Villa near Vienna Project, Outside Vienna, Aus...",Emil Hoppe,7605,"(Austrian, 1876–1957)",(Austrian),(1876),(1957),(Male),1903,"Graphite, pen, color pencil, ink, and gouache ...",...,http://www.moma.org/media/W1siZiIsIjk4Il0sWyJw...,NaN,NaN,NaN,34.3000,NaN,NaN,31.8000,NaN,NaN
3,"The Manhattan Transcripts Project, New York, N...",Bernard Tschumi,7056,"(French and Swiss, born Switzerland 1944)",(),(1944),(0),(Male),1980,Photographic reproduction with colored synthet...,...,http://www.moma.org/media/W1siZiIsIjEyNCJdLFsi...,NaN,NaN,NaN,50.8000,NaN,NaN,50.8000,NaN,NaN
4,"Villa, project, outside Vienna, Austria, Exter...",Emil Hoppe,7605,"(Austrian, 1876–1957)",(Austrian),(1876),(1957),(Male),1903,"Graphite, color pencil, ink, and gouache on tr...",...,http://www.moma.org/media/W1siZiIsIjEyNiJdLFsi...,NaN,NaN,NaN,38.4000,NaN,NaN,19.1000,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132398,An Episode in the Life of a Landscape Painter ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2006,Book from a multiple of archival carrying case...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
132399,Incense Sweaters & Ice (Installation),Martine Syms,68036,"(American, 1988)",(American),(1988),(0),(),2017,"Video (color, sound)",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4500.0
132400,Just For You,Ken Friedman,2007,"(American, born 1949)",(American),(1939),(0),(Male),1967,NaN,...,NaN,NaN,0.0,NaN,0.0000,NaN,NaN,0.0000,NaN,NaN
132401,Marey and Demeny Chronophotographs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1892–1900,"35mm film (black and white, silent)",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,420.0


# Evaluación de datos

In [3]:
artist = list(df.Artist.unique())

In [4]:
len(artist)

13863

In [5]:
df.shape

(132403, 29)

In [6]:
df.isnull().sum()

Title                     45
Artist                  1461
ConstituentID           1461
ArtistBio               4734
Nationality             1461
BeginDate               1461
EndDate                 1461
Gender                  1461
Date                    2166
Medium                 11225
Dimensions             11243
CreditLine              2924
AccessionNumber            0
Classification             0
Department                 0
DateAcquired            5994
Cataloged                  0
ObjectID                   0
URL                    57280
ThumbnailURL           68263
Circumference (cm)    132393
Depth (cm)            120479
Diameter (cm)         130985
Height (cm)            18765
Length (cm)           131665
Weight (kg)           132110
Width (cm)             19664
Seat Height (cm)      132403
Duration (sec.)       129258
dtype: int64

Trabajo con los artistas que han presentado trabajos en los últimos 50 años.

# CLEANING

In [7]:
# DROP columns that are not useful for my analysis.

df1 = df.drop(["Title", "ConstituentID", "ArtistBio","Medium","ThumbnailURL", "BeginDate", "EndDate", "Circumference (cm)", "Depth (cm)", "Diameter (cm)", "Height (cm)", "Length (cm)", "Weight (kg)", "Width (cm)", "Seat Height (cm)", "Duration (sec.)", "Dimensions", "CreditLine", "AccessionNumber", "Department", "DateAcquired", "Cataloged", "ObjectID", "URL"],axis=1)

In [8]:
df1.sample()

,Artist,Nationality,Gender,Date,Classification
21984,Marc Chagall,(French),(Male),1941-1948,Illustrated Book


In [9]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 132403 entries, 0 to 132402
Data columns (total 5 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   Artist          130942 non-null  object
 1   Nationality     130942 non-null  object
 2   Gender          130942 non-null  object
 3   Date            130237 non-null  object
 4   Classification  132403 non-null  object
dtypes: object(5)
memory usage: 5.1+ MB


In [10]:
# Cleaning "date" column to obtain an integer year.
date = list(df1.Date.unique())

In [11]:
date

['1896',
 '1987',
 '1903',
 '1980',
 '1976-77',
 '1968',
 '1900',
 '1978',
 '1905',
 '1906',
 '1979',
 '1980-81',
 '1918',
 '1970',
 '1975',
 '1984',
 '1986',
 '1974',
 'n.d.',
 'c. 1917',
 '1917',
 '1923',
 'Unknown',
 '1930',
 '1936',
 '1935',
 '1937',
 '1938',
 '1977',
 '1958',
 '1985',
 '1989',
 '1949',
 '1958–1964',
 'c. 1935',
 '1991',
 '1941',
 '1965',
 '1981',
 '1983',
 '1985–1988',
 'c. 1989-91',
 '1992',
 '1915-17',
 '1915–1917',
 'c. 1915-17',
 '1953',
 '1910',
 'c.1985',
 '1982–1986',
 '1982-86',
 '1945',
 '1985.',
 '1986.',
 '1923–1924',
 '.1-3 1987; .4 1990',
 '1990',
 '1976',
 '1995',
 '1927–1931',
 'c. 1929-30',
 '1964',
 '1959',
 'c. 1918-20',
 'c.1918-1920',
 '1939',
 'c.1976',
 '1975-79',
 '1993',
 '1989.',
 '1996',
 '1988',
 '1982-83',
 '1982–1983',
 '1952-53',
 '1921',
 '1957',
 '1972',
 '1956-57',
 '1924',
 '1962',
 '1925',
 '1960',
 '1969',
 '1963',
 '1994',
 ' 1961',
 '1960-61',
 '1952',
 'c. 1978-84',
 '1927',
 '1979–1985',
 'before 1933',
 '1961',
 '1929',
 'c

In [12]:
df1['year'] = df1['Date'].str.extract(r"(\d+)")

In [13]:
df1.head()

,Artist,Nationality,Gender,Date,Classification,year
0,Otto Wagner,(Austrian),(Male),1896,Architecture,1896
1,Christian de Portzamparc,(French),(Male),1987,Architecture,1987
2,Emil Hoppe,(Austrian),(Male),1903,Architecture,1903
3,Bernard Tschumi,(),(Male),1980,Architecture,1980
4,Emil Hoppe,(Austrian),(Male),1903,Architecture,1903


In [14]:
# Dropping column "Date"; I use column "year".
df1 = df1.drop(["Date"],axis=1)

In [15]:
df1.sample()

,Artist,Nationality,Gender,Classification,year
55840,Lee Friedlander,(American),(Male),Photograph,1960


In [16]:
# Drop rows that have NaN in year column.
df1 = df1[df1['year'].notna()]

In [17]:
df1['year'] = df1['year'].astype(int)

In [18]:
# I focus on the exhibitions from the last 50 years.
df2 = df1[df1["year"]>=1972]

In [19]:
df2.shape

(42319, 5)

In [20]:
# Elimino los NaN en la columna artist.
df2 = df2[df2['Artist'].notna()]

In [21]:
artist = list(df2.Artist)

In [22]:
artist

['Christian de Portzamparc',
 'Bernard Tschumi',
 'Bernard Tschumi',
 'Bernard Tschumi',
 'Bernard Tschumi',
 'Bernard Tschumi',
 'Bernard Tschumi',
 'Bernard Tschumi',
 'Bernard Tschumi',
 'Bernard Tschumi',
 'Bernard Tschumi',
 'Bernard Tschumi',
 'Bernard Tschumi',
 'Bernard Tschumi',
 'Bernard Tschumi',
 'Bernard Tschumi',
 'Bernard Tschumi',
 'Bernard Tschumi',
 'Bernard Tschumi',
 'Bernard Tschumi',
 'Bernard Tschumi',
 'Bernard Tschumi',
 'Bernard Tschumi',
 'Bernard Tschumi',
 'Bernard Tschumi',
 'Bernard Tschumi',
 'Bernard Tschumi',
 'Bernard Tschumi',
 'Bernard Tschumi',
 'Bernard Tschumi',
 'Bernard Tschumi',
 'Bernard Tschumi',
 'Bernard Tschumi',
 'Bernard Tschumi',
 'Bernard Tschumi',
 'Bernard Tschumi',
 'Bernard Tschumi',
 'Bernard Tschumi',
 'Bernard Tschumi',
 'Bernard Tschumi',
 'Bernard Tschumi',
 'Bernard Tschumi',
 'Bernard Tschumi',
 'Bernard Tschumi',
 'Bernard Tschumi',
 'Bernard Tschumi',
 'Bernard Tschumi',
 'Bernard Tschumi',
 'Bernard Tschumi',
 'Bernard T

In [23]:
#LIMPIEZA columna Artist.
df2['artist_2'] = df2['Artist'].str.split(",")

In [24]:
df2.sample()

,Artist,Nationality,Gender,Classification,year,artist_2
130141,"Hans Danuser, Peter Zumthor",(Swiss) (Swiss),(Male) (Male),Architecture,1998,"[Hans Danuser, Peter Zumthor]"


In [25]:
art = list(df2.artist_2)

In [26]:
art

[['Christian de Portzamparc'],
 ['Bernard Tschumi'],
 ['Bernard Tschumi'],
 ['Bernard Tschumi'],
 ['Bernard Tschumi'],
 ['Bernard Tschumi'],
 ['Bernard Tschumi'],
 ['Bernard Tschumi'],
 ['Bernard Tschumi'],
 ['Bernard Tschumi'],
 ['Bernard Tschumi'],
 ['Bernard Tschumi'],
 ['Bernard Tschumi'],
 ['Bernard Tschumi'],
 ['Bernard Tschumi'],
 ['Bernard Tschumi'],
 ['Bernard Tschumi'],
 ['Bernard Tschumi'],
 ['Bernard Tschumi'],
 ['Bernard Tschumi'],
 ['Bernard Tschumi'],
 ['Bernard Tschumi'],
 ['Bernard Tschumi'],
 ['Bernard Tschumi'],
 ['Bernard Tschumi'],
 ['Bernard Tschumi'],
 ['Bernard Tschumi'],
 ['Bernard Tschumi'],
 ['Bernard Tschumi'],
 ['Bernard Tschumi'],
 ['Bernard Tschumi'],
 ['Bernard Tschumi'],
 ['Bernard Tschumi'],
 ['Bernard Tschumi'],
 ['Bernard Tschumi'],
 ['Bernard Tschumi'],
 ['Bernard Tschumi'],
 ['Bernard Tschumi'],
 ['Bernard Tschumi'],
 ['Bernard Tschumi'],
 ['Bernard Tschumi'],
 ['Bernard Tschumi'],
 ['Bernard Tschumi'],
 ['Bernard Tschumi'],
 ['Bernard Tschumi'],
 

In [27]:
df2['artist_2'] = df2['Artist'].str.split(",")

In [28]:
df2['country'] = df2['Nationality'].str.split(" ")

In [29]:
df2

,Artist,Nationality,Gender,Classification,year,artist_2,country
1,Christian de Portzamparc,(French),(Male),Architecture,1987,[Christian de Portzamparc],[(French)]
3,Bernard Tschumi,(),(Male),Architecture,1980,[Bernard Tschumi],[()]
5,Bernard Tschumi,(),(Male),Architecture,1976,[Bernard Tschumi],[()]
6,Bernard Tschumi,(),(Male),Architecture,1976,[Bernard Tschumi],[()]
7,Bernard Tschumi,(),(Male),Architecture,1976,[Bernard Tschumi],[()]
...,...,...,...,...,...,...,...
132308,Christian Vinck Henriquez,(Venezuelan),(),Print,2016,[Christian Vinck Henriquez],[(Venezuelan)]
132309,Christian Vinck Henriquez,(Venezuelan),(),Multiple,2016,[Christian Vinck Henriquez],[(Venezuelan)]
132331,Laura Owens,(American),(Female),Illustrated Book,2015,[Laura Owens],[(American)]
132332,Laura Owens,(American),(Female),Illustrated Book,2015,[Laura Owens],[(American)]


In [30]:
#LIMPIEZA columna country.
count = list(df2.country)
count

[['(French)'],
 ['()'],
 ['()'],
 ['()'],
 ['()'],
 ['()'],
 ['()'],
 ['()'],
 ['()'],
 ['()'],
 ['()'],
 ['()'],
 ['()'],
 ['()'],
 ['()'],
 ['()'],
 ['()'],
 ['()'],
 ['()'],
 ['()'],
 ['()'],
 ['()'],
 ['()'],
 ['()'],
 ['()'],
 ['()'],
 ['()'],
 ['()'],
 ['()'],
 ['()'],
 ['()'],
 ['()'],
 ['()'],
 ['()'],
 ['()'],
 ['()'],
 ['()'],
 ['()'],
 ['()'],
 ['()'],
 ['()'],
 ['()'],
 ['()'],
 ['()'],
 ['()'],
 ['()'],
 ['()'],
 ['()'],
 ['()'],
 ['()'],
 ['()'],
 ['()'],
 ['()'],
 ['()'],
 ['()'],
 ['()'],
 ['(American)', '()'],
 ['(Dutch)', '(Dutch)'],
 ['(American)'],
 ['()'],
 ['(American)'],
 ['()'],
 ['(American)'],
 ['(American)'],
 ['(American)'],
 ['(American)'],
 ['(American)'],
 ['(Italian)', '(Italian)', '(Italian)'],
 ['(American)'],
 ['(American)'],
 ['(American)'],
 ['(American)'],
 ['(American)'],
 ['(American)'],
 ['(American)'],
 ['(American)'],
 ['(American)'],
 ['(American)'],
 ['(American)'],
 ['(American)'],
 ['(American)'],
 ['(American)'],
 ['(British)'],
 ['(Ameri

In [31]:
df4 = df2.explode('artist_2').explode('country')

In [32]:
count = list(df4.country)
count

['(French)',
 '()',
 '()',
 '()',
 '()',
 '()',
 '()',
 '()',
 '()',
 '()',
 '()',
 '()',
 '()',
 '()',
 '()',
 '()',
 '()',
 '()',
 '()',
 '()',
 '()',
 '()',
 '()',
 '()',
 '()',
 '()',
 '()',
 '()',
 '()',
 '()',
 '()',
 '()',
 '()',
 '()',
 '()',
 '()',
 '()',
 '()',
 '()',
 '()',
 '()',
 '()',
 '()',
 '()',
 '()',
 '()',
 '()',
 '()',
 '()',
 '()',
 '()',
 '()',
 '()',
 '()',
 '()',
 '()',
 '(American)',
 '()',
 '(American)',
 '()',
 '(Dutch)',
 '(Dutch)',
 '(Dutch)',
 '(Dutch)',
 '(American)',
 '()',
 '(American)',
 '()',
 '(American)',
 '(American)',
 '(American)',
 '(American)',
 '(American)',
 '(Italian)',
 '(Italian)',
 '(Italian)',
 '(Italian)',
 '(Italian)',
 '(Italian)',
 '(Italian)',
 '(Italian)',
 '(Italian)',
 '(American)',
 '(American)',
 '(American)',
 '(American)',
 '(American)',
 '(American)',
 '(American)',
 '(American)',
 '(American)',
 '(American)',
 '(American)',
 '(American)',
 '(American)',
 '(American)',
 '(British)',
 '(American)',
 '(American)',
 '(American

In [104]:
df4['Country'] = df4['country'].replace("(French)", "France").replace("(American)", "United States").replace("(Italian)", "Italy").replace("(Dutch)", "Netherlands").replace("(British)", "United Kingdom").replace("(Japanese)", "Japan").replace("(Argentine)", "Argentina").replace("(Swiss)", "Switzerland").replace("(Luxembourgish)", "Luxembourg").replace("(Spanish)", "Spain").replace("(Polish)", "Poland").replace("(Austrian)", "Austria").replace("(Iranian)", "Iran").replace("(German)", "Germany").replace("(Moroccan)", "Morocco").replace("(Danish)", "Denmark").replace("(Canadian)", "Canada").replace("(Brazilian)", "Brazil").replace("(Venezuelan)", "Venezuela").replace("(Belgian)", "Belgium").replace("(Norwegian)", "Norway").replace("(Finnish)", "Finland").replace("(Swedish)", "Sweden").replace("(Colombian)", "Colombia").replace("(Australian)", "Autralia").replace("(Yugoslav)", "Yugoslavia").replace("(Nationality", "None").replace("unknown)", "None").replace("(Hungarian)", "Hungary").replace("(Mexican)", "Mexico").replace("(Greek)", "Greece").replace("(Israeli)", "Israel").replace("(Croatian)", "Croatia").replace("(Cuban)", "Cuba").replace("(Thai)", "Thailand"). replace("(Czech)", "Czech Republic").replace("(Chilean)", "Chile").replace("(Various)", "None").replace("(Romanian)", "Romania").replace("(South", "South Africa").replace("African)", "South Africa").replace("(Russian)", "Russian Federation").replace("(Congolese)", "Central African Republic").replace("(Ukrainian)", "Ukraine").replace("(Peruvian)", "Peru").replace("(Indian)", "India").replace("(Cambodian)", "Cambodia").replace("(Haitian)", "Haiti").replace("(Scottish)", "United Kingdom").replace("(Korean)", "South Korea").replace("(Slovak)", "Slovak Republic").replace("(Estonian)", "Estonia").replace("(Pakistani)", "Pakistan").replace("(Icelandic)", "Iceland").replace("(Portuguese)", "Portugal").replace("(Chinese)", "China").replace("(Paraguayan)", "Paraguay").replace("(Uruguayan)", "Uruguay").replace("(Tunisian)", "Tunisia").replace("(Guyanese)", "Guatemala").replace("(Senegalese)", "Senegal").replace("(Bahamian)", "Cuba").replace("(Turkish)", "Turkey").replace("(Malian)", "Mali").replace("(Bulgarian)", "Bulgaria").replace("(New", "New Zealand").replace("Zealander)", "New Zealand").replace("(Irish)", "Ireland").replace("(Lebanese)", "Lebanon").replace("(English)", "United Kingdom"). replace("(Welsh)", "United Kingdom").replace("(Cypriot)", "Cyprus").replace("(Kenyan)", "Kenya").replace("(Syrian)", "Syrian Arab Republic").replace("(Saudi", "Saudi Arabia").replace("Arabian)", "Saudi Arabia").replace("(Slovenian)", "Slovenia").replace("(Nigerian)", "Central African Republic").replace("(Native", "United States").replace("American)", "United States").replace("(Bosnian)","Bosnia and Herzegovina").replace("Unknown)", "None").replace("(Kazakhstani)", "Kazakhstan").replace("(Kyrgyzstani)", "Kyrgyz Republic").replace("(Palestinian)", "Israel").replace("(Catalan)", "Spain").replace("(Vietnamese)", "Vietnam").replace("(Indonesian)", "Indonesia").replace("(Filipino)", "Philippines").replace("(Guatemalan)", "Guatemala").replace("(Egyptian)", "Egypt").replace("(Ghanaian)", "Ghana").replace("(Serbian)", "Bosnia and Herzegovina").replace("(Mauritanian)", "Mauritania").replace("(Ugandan)", "Uganda").replace("(Taiwanese)", "Taiwan").replace("(Albanian)", "Albania").replace("(Malaysian)", "Malaysia").replace("(Iraqi)", "Iraq").replace("()", "None").replace("(Tajik)", "Tajikistan").replace("(Singaporean)", "Singapore").replace("(Puerto", "Dominican Republic").replace("Rican)","Dominican Republic")

In [105]:
country2 = list((df4['Country']).unique())

In [106]:
country2

['France',
 'None',
 'United States',
 'Netherlands',
 'Italy',
 'United Kingdom',
 'Japan',
 'Argentina',
 'Switzerland',
 'Luxembourg',
 'Spain',
 'Austria',
 'Poland',
 'Iran',
 'Germany',
 'Canada',
 'Belgium',
 'Norway',
 'Brazil',
 'Finland',
 'Denmark',
 'Sweden',
 'Morocco',
 'Colombia',
 'Autralia',
 'Yugoslavia',
 'Hungary',
 'Mexico',
 'Greece',
 'Israel',
 'Croatia',
 'Cuba',
 'Chile',
 'Russian Federation',
 'Thailand',
 'Czech Republic',
 'Romania',
 'South Africa',
 'Central African Republic',
 'Ukraine',
 'Peru',
 'India',
 'Cambodia',
 'Venezuela',
 'Haiti',
 'South Korea',
 'Slovak Republic',
 'Estonia',
 'Pakistan',
 'Iceland',
 'Portugal',
 'China',
 'Paraguay',
 'Uruguay',
 'Tunisia',
 'Guatemala',
 'Senegal',
 'Turkey',
 'Mali',
 'Bulgaria',
 'New Zealand',
 'Ireland',
 'Lebanon',
 'Cyprus',
 'Kenya',
 'Taiwan',
 'Slovenia',
 '(Georgian)',
 'Egypt',
 'Albania',
 'Tajikistan',
 '(Algerian)',
 '(Lithuanian)',
 '(Namibian)',
 'Bosnia and Herzegovina',
 'Ghana',
 '(Af

In [85]:
country = list((df4['country']).unique())

In [37]:
country

['(French)',
 '()',
 '(American)',
 '(Dutch)',
 '(Italian)',
 '(British)',
 '(Japanese)',
 '(Argentine)',
 '(Swiss)',
 '(Luxembourgish)',
 '(Spanish)',
 '(Austrian)',
 '(Polish)',
 '(Iranian)',
 '(German)',
 '(Canadian)',
 '(Belgian)',
 '(Norwegian)',
 '(Brazilian)',
 '(Finnish)',
 '(Danish)',
 '(Swedish)',
 '(Moroccan)',
 '(Colombian)',
 '(Australian)',
 '(Yugoslav)',
 '(Nationality',
 'unknown)',
 '(Hungarian)',
 '(Mexican)',
 '(Greek)',
 '(Israeli)',
 '(Croatian)',
 '(Cuban)',
 '(Chilean)',
 '(Various)',
 '(Russian)',
 '(Thai)',
 '(Czech)',
 '(Romanian)',
 '(South',
 'African)',
 '(Congolese)',
 '(Ukrainian)',
 '(Peruvian)',
 '(Indian)',
 '(Cambodian)',
 '(Venezuelan)',
 '(Haitian)',
 '(Scottish)',
 '(Korean)',
 '(Slovak)',
 '(Estonian)',
 '(Pakistani)',
 '(Icelandic)',
 '(Portuguese)',
 '(Chinese)',
 '(Paraguayan)',
 '(Uruguayan)',
 '(Tunisian)',
 '(Guyanese)',
 '(Senegalese)',
 '(Bahamian)',
 '(Turkish)',
 '(Malian)',
 '(Bulgarian)',
 '(New',
 'Zealander)',
 '(Irish)',
 '(Lebanese

In [38]:
df4

,Artist,Nationality,Gender,Classification,year,artist_2,country,Country
1,Christian de Portzamparc,(French),(Male),Architecture,1987,Christian de Portzamparc,(French),France
3,Bernard Tschumi,(),(Male),Architecture,1980,Bernard Tschumi,(),()
5,Bernard Tschumi,(),(Male),Architecture,1976,Bernard Tschumi,(),()
6,Bernard Tschumi,(),(Male),Architecture,1976,Bernard Tschumi,(),()
7,Bernard Tschumi,(),(Male),Architecture,1976,Bernard Tschumi,(),()
...,...,...,...,...,...,...,...,...
132308,Christian Vinck Henriquez,(Venezuelan),(),Print,2016,Christian Vinck Henriquez,(Venezuelan),Venezuela
132309,Christian Vinck Henriquez,(Venezuelan),(),Multiple,2016,Christian Vinck Henriquez,(Venezuelan),Venezuela
132331,Laura Owens,(American),(Female),Illustrated Book,2015,Laura Owens,(American),United States
132332,Laura Owens,(American),(Female),Illustrated Book,2015,Laura Owens,(American),United States


In [39]:
# df5 = df4.groupby(["artist_2", 'country'])

In [40]:
df5.head()

NameError: name 'df5' is not defined

In [ ]:
df4

In [ ]:
country = list((df4.Country).unique())

In [ ]:
country

In [ ]:
df2 = df2[df2['Artist'].notna()]

In [ ]:
df2 = df2[df2['Artist'].notna()]

In [ ]:
df2.info()

In [ ]:
df3 = df2.drop(df2[df2['Nationality']=="()"].index)

In [ ]:
df3

In [ ]:
df['Art'] = df.Artist.apply(lambda s: [s[:i] for i in range(1, len(s)+1, 2)])

In [ ]:
df4

# Enrich data base with scraping

EDUCATIONAL DATA (Creativity index)

In [44]:
url = 'http://chartsbin.com/view/41109'

In [45]:
html=req.get(url).content

In [46]:
soup=bs(html, 'html.parser')
type(soup)

bs4.BeautifulSoup

In [47]:
tabla=soup.find('table')

type(tabla)

bs4.element.Tag

In [48]:
filas=tabla.find_all('tr')

elementos=[[e.text for e in fila] for fila in filas]

elementos

[['Rank',
  'Country',
  'Technology',
  'Talent',
  'Tolerance',
  'Global Creativity Index'],
 ['1', 'Australia', '7', '1', '4', '0.97'],
 ['2', 'United States', '4', '3', '11', '0.95'],
 ['3', 'New Zealand', '7', '8', '3', '0.949'],
 ['4', 'Canada', '13', '14', '1', '0.92'],
 ['5', 'Denmark', '10', '6', '13', '0.917'],
 ['5', 'Finland', '5', '3', '20', '0.917'],
 ['7', 'Sweden', '11', '8', '10', '0.915'],
 ['8', 'Iceland', '26', '2', '2', '0.913'],
 ['9', 'Singapore', '7', '5', '23', '0.896'],
 ['10', 'Netherlands', '20', '11', '6', '0.889'],
 ['11', 'Norway', '18', '12', '9', '0.883'],
 ['12', 'United Kingdom', '15', '20', '5', '0.881'],
 ['13', 'Ireland', '23', '21', '7', '0.845'],
 ['14', 'Germany', '7', '28', '18', '0.837'],
 ['16', 'Switzerland', '19', '22', '17', '0.822'],
 ['16', 'France', '16', '26', '16', '0.822'],
 ['16', 'Slovenia', '17', '8', '35', '0.822'],
 ['18', 'Belgium', '28', '18', '14', '0.817'],
 ['19', 'Spain', '31', '19', '12', '0.811'],
 ['20', 'Austria', '12

ECONOMY

In [ ]:
url = 'https://datosmacro.expansion.com/pib?anio=1999'

In [ ]:
html=req.get(url).content

In [ ]:
soup=bs(html, 'html.parser')
type(soup)

In [ ]:
tabla=soup.find('table')

type(tabla)

In [ ]:
filas=tabla.find_all('tr')

elementos=[[e.text.replace("\xa0M€", "").replace("\xa0M$", "").replace("[+]", "").strip() for e in fila] for fila in filas]

elementos[:5]